# Haplotye Aware Edit Distance

## Construct Alignment graph for a given induced subgraph and string

In [3]:
# !python -m pip install -i https://pypi.gurobi.com gurobipy
# !pip install gurobipy  # install gurobipy, if not already installed
# !pip install networkx
# !pip install numpy
# !pip install scipy

In [4]:
import gurobipy as gp  # import the installed package
import networkx as nx
from gurobipy import *
from gurobipy import GRB
import numpy as np
import os 
import scipy.sparse as sp
import time
import math

In [5]:
V = [0,1,2,3,4,5,6,7,8, 9, 10, 11, 12, 13]
E = [(0,1,"A"),(1,2,"A"),(1,2,"G"), (1,2, "C"),(2,3,"A"), (2,4,"e"),\
    (3,4,"C"), (4,5,"A"), (5,6,"A"), (6,7,"C"), (7,8, "T"), (8,9,"T"), (9,10, "G"),\
    (10,11,"T"),(11,9, "C"),(9,12, "T"),(12,13, "A")]

n = len(V)
print("n",n)
alpha = 3
delta = 1                                      
P = [1,2,9]

# List of haplotypes
#K = ["TGACAACTT---TA","TAACAACTTGTCTA","TC--AACTT---TA"]
K = ["TAACAACTTGTCTA","TC--AACTT---TA"] # we need to preserve R1

# Align haplotypes based on R1
for p in P:
    for h in K:
        t = ""
        for i in range(p,p+alpha): #  Because string starts at 0, we ut minus 1
            if h[i] != '-':
                t = t + h[i]
        if len(t) < alpha: # Extend the string if its length is less than alpha
            f = len(t)
            print("f",f)
            print("len t", len(t))
            r = alpha - len(t)
            print("r", r)
            for i in range(p+f+1,p+f+r+2):
                if p+f+1<= n and p+f+r <= n and h[i] != '-':
                    t = t + h[i]


        print("h", h, "p", p,"t", t)
    print("----")


S = "AAC"

n 14
h TAACAACTTGTCTA p 1 t AAC
f 1
len t 1
r 2
h TC--AACTT---TA p 1 t CAA
----
h TAACAACTTGTCTA p 2 t ACA
f 1
len t 1
r 2
h TC--AACTT---TA p 2 t AAAC
----
h TAACAACTTGTCTA p 9 t GTC
f 0
len t 0
r 3
h TC--AACTT---TA p 9 t TA
----


 # preprocessing haplotypes and nodes and edges

In [6]:
def convert_string_toint(S):
    S_int = []
    for item in S:
        if item == "A":
            S_int.append(2)
        if item == "C":
            S_int.append(3)
        if item == "G":
            S_int.append(4)
        if item == "T":
            S_int.append(5)
    return S_int        

In [7]:
S_int = convert_string_toint(S)
print(S_int)

[2, 2, 3]


In [8]:
def convert_edge_lables_toint(E):
    E_int = []
    for edge in E:
        # A:2, C:3, G: 4, T:5, e:6
        if edge[2] == "A":
            E_int.append((edge[0], edge[1], 2))
        if edge[2] == "C":
            E_int.append((edge[0], edge[1], 3))
        if edge[2] == "G":
            E_int.append((edge[0], edge[1], 4))
        if edge[2] == "T":
            E_int.append((edge[0], edge[1], 5)) 
        if edge[2] == "e":
            E_int.append((edge[0], edge[1], 6))  
    return E_int

In [9]:
E_int = convert_edge_lables_toint(E)
print(E_int)

[(0, 1, 2), (1, 2, 2), (1, 2, 4), (1, 2, 3), (2, 3, 2), (2, 4, 6), (3, 4, 3), (4, 5, 2), (5, 6, 2), (6, 7, 3), (7, 8, 5), (8, 9, 5), (9, 10, 4), (10, 11, 5), (11, 9, 3), (9, 12, 5), (12, 13, 2)]


# Find Span for the induced graph

In [10]:
# E: list of edges in the complete variation graph ( vertex1, vertex1, label)
# V : List of nodes in the complete variation graph
# V1 : the variation node that we want to find the span for it

def find_span(v1, delta, alpha, V, E_int):
    
    n = len(V)
    if v1>n:
        print(" The variant position is invalid")
        return 
    

    Active = {} # keep track of number of active nodes (outdegree edges) for each vertex
    number_active = 0  # Shows the number of non zerors s in the active array 
    
    Dist = {}  # An array that shows the distance of each vertex from v1 ( the shortest distance)
    Dist[v1] = 0
    
    leng = alpha + delta

    out_degree = {} # shows the list of out-going nodes for each node
    in_degree = {}  # shows the list of incoming nodes for each node

    out_degree[v1] = {edge[1] for edge in E_int if edge[0] == v1}
    
    if len(out_degree[v1])>0:
        Active[v1] = len(out_degree[v1])
        number_active = 1
#         number_active = np.count_nonzero(Active, axis = 0)
    else:
        return 
    
      
#     print("out_degree", out_degree)
#     print("Active", Active)
#     number_active = np.count_nonzero(Active, axis = 0)
#     print("number_active", number_active)

    i = v1+ 1 
#     print("first i", i)
    min_dist =0

    while i < n and number_active>0 and min_dist < leng:
        print("")
        print("i:", i)
        
        min_dist = math.inf

        in_degree[i] = {edge[0] for edge in E_int if edge[1] == i} # to define set in python 
        out_degree[i] = {edge[1] for edge in E_int if edge[0] == i}
        
#         Active[i] = len(out_degree[i])
#         print("Active", Active)


        for j in in_degree[i]:

            print("j:",j)
            Active[j] = Active[j] -1


#             dist = 0
#             print("Active", Active)
            
            if Active[j] <=0:
                number_active = number_active -1
#                 print("number_Active1", number_active)

            for edge in E_int:
                dist = math.inf
                if (edge[0] == j and edge[1] == i and edge[2] == 6):
                    dist = Dist[j]
#                     print("distance of epsilon", dist)

                if (edge[0] == j and edge[1] == i and edge[2] != 6):
                    dist = Dist[j] +1
#                     print(" chand bar injaae")
#                     print("dist", dist)

                if dist <= min_dist:
                    min_dist = dist
#                     print("dist dakhele min", dist)
                   
#         print("min distance", min_dist)            

        Dist[i] = min_dist
        print("Dist", Dist)


        if(min_dist < leng and len(out_degree)>0):
            Active[i] = len(out_degree[i])
           
            number_active = number_active +1
#             print("number_active", number_active)
#             print("i inja", i)
            i = i +1
            
#         if min_dist ==   math.inf:
#             print("i inf", i)
#             print("i qable inf", i-1)
#             return i-1
     
#         i = i+1
#         print("number_Active2", number_active)

    print(i)
    return i

    

In [11]:
end = find_span(1, delta, alpha, V, E_int)
print(end)


i: 2
j: 1
Dist {1: 0, 2: 1}

i: 3
j: 2
Dist {1: 0, 2: 1, 3: 2}

i: 4
j: 2
j: 3
Dist {1: 0, 2: 1, 3: 2, 4: 1}

i: 5
j: 4
Dist {1: 0, 2: 1, 3: 2, 4: 1, 5: 2}

i: 6
j: 5
Dist {1: 0, 2: 1, 3: 2, 4: 1, 5: 2, 6: 3}

i: 7
j: 6
Dist {1: 0, 2: 1, 3: 2, 4: 1, 5: 2, 6: 3, 7: 4}
7
7


# Construct Alignment grpah for the induced graph

In [12]:
"""
V is a list of vertices (induced graph), 
E is the list of edges (induced graph)
S is the string of lengthh alpha
G_a is the alignment_graph

"""

def alignment_graph(V, E_int, S_int, start_s, start_y):
    E_a =[]
    w_a = []
#     E_a_ins = []
#     w_a_ins = []
    sai = []  # map edge in alignment graph to the corresponding edge in complete variation graph
    y_index = []  # index corresponds to each edge in the E_a that can be used for ILP (node, node, lable charcter, index)
    
    n = len(V) # number of nodes
    print("n", n)
    alpha = len(S)

    # 1 for s and V[n-1]+n*(alpha) +1
    s= start_s
    print(V[n-1]+(n)*(alpha) +1)
    t= V[n-1]+(n)*(alpha) +1
    print("V[n-1]", V[n-1])
    print("t",t)
    V_a =[s]
#     for i in V:
#         for j in range(alpha+1):
#             V_a.append(i+j*n)
#     print("V_a",V_a)
    
    
    for j in range(alpha+1):
        for i in V:
            V_a.append(i+j*n)
    
    
    V_a.append(t)
    print("V_a",V_a)
    
    # Add the edge from the source node to the first node
    E_a.append((1, V[0],1))
    w_a.append(1)
    
    count = 0  # count lables for y
    
    for j in range(alpha): # for each level, we don't need to have it for the last level 
        for edge in E_int:

           # e-edge
            if edge[2] == 6:
                print("e-edge")
                count = count + 1
                E_a.append((edge[0]+j*n,edge[1]+j*n,0))
                w_a.append(0)
                sai.append(((edge[0]+j*n,edge[1]+j*n,0),(edge[0],edge[1],6)))
                y_index.append((edge[0]+j*n,edge[1]+j*n,edge[2],count))

             # Insertion-edge
            if edge[2] != 6:
                print("Insertion")
                count = count + 1
                print((edge[0]+j*n,edge[1]+j*n))
                print("")
                E_a.append((edge[0]+j*n,edge[1]+j*n,1)) 
                w_a.append(1)
                sai.append(((edge[0]+j*n,edge[1]+j*n,1),(edge[0],edge[1],edge[2])))
                y_index.append((edge[0]+j*n,edge[1]+j*n,edge[2],count))
               

            # Exact match-edge
            if edge[2] == S_int[j]and edge[2] !=6:
                print("Exact")
                count = count + 1
                print((edge[0]+j*n,edge[1]+j*n+n))
                print("")
                E_a.append((edge[0]+j*n,edge[1]+j*n+n,0))
                w_a.append(0)
                sai.append(((edge[0]+j*n,edge[1]+j*n+n,0),(edge[0],edge[1],edge[2])))
                y_index.append((edge[0]+j*n,edge[1]+j*n+n,edge[2],count))
                
            # substitution-edge
            if edge[2] !=S_int[j] and edge[2] !=6:
                print("Substi")
                count = count + 1
                print((edge[0]+j*n,edge[1]+j*n+n))
                print("")
                E_a.append((edge[0]+j*n,edge[1]+j*n+n,1))
                w_a.append(1)
                sai.append(((edge[0]+j*n,edge[1]+j*n+n,1),(edge[0],edge[1],edge[2])))
                y_index.append((edge[0]+j*n,edge[1]+j*n+n, edge[2],count))
                
    # deletion-edge            
    for i in V:  
        for j in range(alpha):
            print("deletion")
            count = count + 1
            print((i+j*n,i+j*n+n))
            print("")
            E_a.append((i+j*n,i+j*n+n,1))
            w_a.append(1)
            y_index.append((i+j*n,i+j*n+n,"e",count))


    print("")
#     E_a = E_a + E_a_ins
#     w_a = w_a + w_a_ins
    
    
    # Add edges the last level to t
    r = V[n-1]+(n*(alpha-1))+1
    for index in V_a[r-1:r+n-1]:
        count = count + 1
        E_a.append((index, t,1))
        w_a.append(1)
        y_index.append((index, t,"t",count))
    
    
    print("new_ea", E_a)
#     E_a.sort()
    print("")
#     E_a.sort()
#     print("new_ea sorted", E_a)
    print("")
    print(" len new_ea sorted", len(E_a))
    
    print("")
    print("w_a", w_a)
    print("length of w_a", len(w_a))
    print("y_index for ILP per edge", y_index)
    print("length of y_index for ILP per edge", len(y_index))
     
    return V_a, E_a, sai, w_a, y_index, len(y_index)


In [13]:
Start = 0
print(V[1:end+1])
V= V[1:end+1]
#V_a, E_a, sai, w_a , y_index, y_index_length= alignment_graph(V, E_int, S_int)
V_a, E_a, sai, w_a , y_index, y_index_length= alignment_graph(V_induced, E_int, S_int, Start)
Start = Start + y_index_length

[1, 2, 3, 4, 5, 6, 7]


NameError: name 'V_induced' is not defined

## Construct ILP constraints

In [14]:
def local_ILP(model, V, V_a, y_index,start_index, alpha, delta ):
#     delta = 1
    alpha = 3
    start_index = 0 # start is an input from the previous one
    m = len(E_a)
    print(m)
    # n = len(V_a)
    n = len(V)

    # starting time
    start = time.time()

    # Create a new mode
#     model = gp.Model()

    #model.params.LogToConsole = LogToConsole

    # Create variables
    # y = model.addMVar(lb=0.0, ub=GRB.INFINITY,shape=m, vtype=GRB.BINARY, name="y")
    y = model.addMVar(lb=0.0, ub=GRB.INFINITY,shape=m, vtype=GRB.CONTINUOUS, name="y")

    # Add constraints
    offset = start_index 
    rhs = gp.LinExpr(0)

    # Add constraints for the first node   
    node = V_a[1]
    search_list = [a_tuple[3] for (index, a_tuple) in enumerate(y_index) if a_tuple[0]== node]
    for item in search_list:
        rhs += y[item + offset] 
        print(item + offset)
        print (" ")
    model.addConstr(1 == rhs)  


    # We define thre constraints for all nodes except the last row

    # Find the last node index in the level before the last level
    b = n*(alpha-1) + n

    for node in V_a[2:(b+1)]:
        print (" ")
        print("node", node)
        lhs = gp.LinExpr(0)  
        rhs = gp.LinExpr(0)

        # indegree
        search_list_in_deg = [a_tuple[3] for (index, a_tuple) in enumerate(y_index) if a_tuple[1]== node]
        for item in search_list_in_deg:
            lhs += y[item + offset]
            print("indegree",item + offset)

        # outdegree
        search_list_out_deg = [a_tuple[3] for (index, a_tuple) in enumerate(y_index) if a_tuple[0]== node]
        for item in search_list_out_deg:
            rhs += y[item + offset ]
            print("outdegree",item + offset)



       # Add constraints    
        #model.addConstr(lhs == rhs) 
        model.addConstr(lhs -rhs == 0)

    # Add constraints for the last node 
    node = V_a[-1]
    search_list_in = [a_tuple[3] for (index, a_tuple) in enumerate(y_index) if a_tuple[1]== node]
    lhs = gp.LinExpr(0)  
    for item in search_list_in:
        lhs += y[item + offset]
        print(item + offset)

    model.addConstr(lhs == 1) 
    
    
    # Add constraints biniing 
    x_y_corres = []
    for edge_y in y_index:
        for edge_x in x_index:
            for j in range(alpha):
                if ((edge_y[0] == edge_x[0]+j*n) and (edge_y[1] == edge_x[1]+j*n) and (edge_y[2] == edge_x[2])):
                    x_y_corres.append((edge_x[3],edge_y[3]+j*n))
    #                 model.addConstr(x[edge_x[3]]+y[edge_y[3]+j*n] , GRB.LESS_EQUAL, 1.0 )
                    model.addConstr(x[edge_x[3]]+y[edge_y[3]+j*n] , GRB.LESS_EQUAL, 1.0 )


    # set objective function
    obj = gp.LinExpr(0)
    for i in range(m):
        obj += w_a[i]*y[i]
    model.addConstr(obj <= delta)

#     # set objective function
#     model.setObjective(obj, GRB.MINIMIZE)

#     #Optimize model
#     model.optimize()
    print (model.display()) # print constraints Gurobi Python

# #     total_y = model.objVal
#     edge_selected_to_removed = [i for i in range(m) if y[i].x == 0]
#     edges_selected_to_retain= [i for i in range(m) if y[i].x  == 1]
#     y_value = []
#     y_value =[ y[i].x for i in range(m)] 

#     for i in range(m):
#         print("y[",i,"].x",y[i].x)
#     print("y_value", y_value)    

#     print("total_y",total_y)
#     print(" Edge_selected_to_removed",edge_selected_to_removed )
#     print(" ")
#     print(" Edges_selected_to_retain", edges_selected_to_retain)



#     # model.write(file_path)

# #     print('obj: %g' % model.objVal)
#     # for item in y:
#     #     print(item.y)

#     # end time
#     end = time.time()
#     total_time_ILP = end - start
#     print("Total time:", total_time_ILP)
    
    return y_value, y,edges_selected_to_retain


In [15]:
start_index = 1

In [16]:
# # P is varinat position
# # K is the list of haplotypes
'''
V: is the list of vertices ( complte variation graph)
E: list of edges ( complete vg)
P : variant poisions
K: list of haplotypes

'''

def global_ILP(V, E, alpha, delta, P,K):
    
    E_int = convert_edge_lables_toint(E)

    # Number of x decision variables
    N = len(E_int)
    x_index = []  # x index for ILP
    
    count = 0 
    
    for edge in E_int:
        x_index.append((edge[0], edge[1], edge[2], count))  # edge[2] is lable 
        count = count + 1

    print(x_index)    

    # starting time
    start = time.time()

    # Create a new mode
    model = gp.Model()

    #model.params.LogToConsole = LogToConsole

    # Create variables

    x = model.addMVar(lb=0.0, ub=GRB.INFINITY,shape=N+1, vtype=GRB.BINARY, name="x")

    for p in P:
       
        end = find_span(p, delta, alpha, V, E_int)
        V = V[p-1: end]
       
    
#         print( "Span for position", p, "for haplotype", h,"is equal:", end)
        for h in K:
            
            # string corresponds to haplotype
            S = ""
            for i in range(p,p+alpha):
                if h[i] != '-':
                    S = S + h[i]
            if len(S) < alpha: # Extend the string if its length is less than alpha
                r = alpha - len(S)
                if len(S)+r <= n:
                    S = S + h[p+alpha:p+alpha+r]
        
#             S = h[index][h:end] # string corresponds to haplotype
            
            
            S_int = convert_string_toint(S)
            # Find span should be called here
            # 
            V_a, E_a, sai, w_a , y_index, y_index_length= alignment_graph(V, E_int, S_int)
            
            y_value,y,edges_selected_to_retain = local_ILP(model, V, V_a, y_index, start_index, alpha, delta )

            for i in range(len(y_value)):
                print("y_value[",i,"]",y_value[i])
                

            # set objective function
            obj = gp.LinExpr(0)
            for i in range(N+1):
                obj += x[i]

            # maximize c.x
            model.setObjective(obj, GRB.MAXIMIZE)

            # model.addConstr(lhs , GRB.LESS_EQUAL, 1.0 * delta)


            # Optimize model
            model.optimize()
            print (model.display()) # print constraints Gurobi Python

#             total_y = model.objVal
            # yedge_selected_to_removed = [i for i in range(m) if y[i].x == 0]
            # yedges_selected_to_retain= [i for i in range(m) if y[i].x  == 1]

            xedge_selected_to_removed = [i for i in range(N) if x[i].x == 1]
            xedges_selected_to_retain= [i for i in range(N) if x[i].x  == 0]


            # for i in range(m):
            #     print("y[",i,"].x",y[i].x)

            for i in range(N+1):    
                print("x[",i,"].x",x[i].x)


            # print(" yEdge_selected_to_removed",yedge_selected_to_removed )
            # print(" ")
            # print(" yEdges_selected_to_retain", yedges_selected_to_retain)

            print(" xEdge_selected_to_removed",xedge_selected_to_removed )
            print(" ")
            print(" xEdges_selected_to_retain", xedges_selected_to_retain)


            # model.write(file_path)

            print('obj: %g' % model.objVal)
            # for item in y:
            #     print(item.y)

            # end time
            end = time.time()
            total_time_ILP = end - start
            print("Total time:", total_time_ILP)

In [17]:
global_ILP(V, E, alpha, delta, P,K)

[(0, 1, 2, 0), (1, 2, 2, 1), (1, 2, 4, 2), (1, 2, 3, 3), (2, 3, 2, 4), (2, 4, 6, 5), (3, 4, 3, 6), (4, 5, 2, 7), (5, 6, 2, 8), (6, 7, 3, 9), (7, 8, 5, 10), (8, 9, 5, 11), (9, 10, 4, 12), (10, 11, 5, 13), (11, 9, 3, 14), (9, 12, 5, 15), (12, 13, 2, 16)]
Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-08

i: 2
j: 1
Dist {1: 0, 2: 1}

i: 3
j: 2
Dist {1: 0, 2: 1, 3: 2}

i: 4
j: 2
j: 3
Dist {1: 0, 2: 1, 3: 2, 4: 1}

i: 5
j: 4
Dist {1: 0, 2: 1, 3: 2, 4: 1, 5: 2}

i: 6
j: 5
Dist {1: 0, 2: 1, 3: 2, 4: 1, 5: 2, 6: 3}
7


TypeError: alignment_graph() missing 2 required positional arguments: 'start_s' and 'start_y'